- add products (more scalable)
    - user input: ask user to specify # ofproducts, expiry dates. perishable or non
- change time frame
    - issue with 7 days (update inventory function)
-

In [4]:
import numpy as np

In [5]:
def mod_pert_random(low, likely, high, confidence=4, samples=1):
    """Produce random numbers according to the 'Modified PERT' distribution.
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        Currently allows values 1-18
    :param samples: Number of samples
    I got this function from Professor Weible's class examples :
    https://github.com/iSchool-597PR/Examples_2021Fall/blob/main/week_07/Probability_Distributions.ipynb
    """
    # Check for reasonable confidence levels to allow:
    if confidence < 1 or confidence > 18:
        raise ValueError('confidence value must be in range 1-18.')

    mean1 = (low + confidence * likely + high) / (confidence + 2)

    a = (mean1 - low) / (high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)

    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    beta = [int(x) for x in beta]
    return beta

In [72]:
class Product:
    def __init__(self, name, expiry_time, qty, demand_upper_bound, demand_lower_bound, storage_capacity, profit_margin, storage_cost_per_day, price_sold, restock_option, simulation_time):
        # input parameters
        self.name = name
        self.expiry_time = expiry_time
        self.qty = qty
        self.storage_capacity = storage_capacity
        self.profit_margin = profit_margin
        self.storage_cost_per_day = storage_cost_per_day
        self.price_sold = price_sold
        self.demand_upper_bound = demand_upper_bound
        self.demand_lower_bound = demand_lower_bound
        self.simulation_time = simulation_time
        self.restock_option = restock_option

        # computation parameters
        self.daily_demand_list = []
        self.weekly_demand_dict = {}


    # weeks in simualtion
    def weeks_in_simulation_time(self):
        self.num_weeks = self.simulation_time//7
        return self.num_weeks


    def weekly_storage_cost(self):
        self.weekly_storage_cost = 7 * self.storage_cost_per_day
        return self.weekly_storage_cost

    # demand distribution for each product
    def demand_distribution(self):
        mean_demand_bound = round( ((self.demand_upper_bound + self.demand_lower_bound)/2),2)
        daily_demand = (mod_pert_random(self.demand_lower_bound * self.qty, mean_demand_bound * self.qty, self.demand_upper_bound * self.qty, samples=self.simulation_time))
        self.daily_demand_list.append(daily_demand)
        return self.daily_demand_list

    # defective items
    def defective_items(self):
        percent_of_defective = np.random.choice(list(range(5, 11)))                # Selects a random % value between 5 to 10
        defective_items = (self.qty * percent_of_defective) // 100                     # Number of defective items during restock (5-10%)
        return defective_items

    def qty_after_restocking(self):
        # restock weekly to full capacity
        if self.restock_option == 1:
            restock_date_list = []
            for i in range(0, self.simulation_time, 7):
                restock_date_list.append(i)
                self.qty = self.storage_capacity

        #restock weekly based on demand
        if self.restock_option == 2:
            weekly_dict = {}
            daily_demand = []
            day_count = 0
            for i in range(self.num_weeks+1):
                daily_demand = (self.demand_distribution()[0])[day_count:day_count+7]
                # total demand, total weekly revenue, total proft, missed profit
                weekly_dict[i] = sum(daily_demand), sum(daily_demand) * self.qty, sum(daily_demand) * self.qty * self.profit_margin * self.price_sold - self.weekly_storage_cost
                day_count += 7


k = Product('iphone', 0, 100, 0.25, 0.20, 1000, 0.8, 20, 50, 1, 10)


In [96]:
# weekly demand dictionary
# def weekly_demand(self):
#     for i in range(self.num_weeks):
#         for j in range(self.daily_demand_list[0]):
#             7_day_demand = (self.daily_demand_list[0])[j:j+7]
#         self.weekly_demand_dict[i] = 7_day_demand
#         j += 7
#     return self.weekly_demand_dict

dict = {}
day_demand = []
j = 0
for i in range(2):
    day_demand = (k.demand_distribution()[0])[j:j+7]
    dict[i] = day_demand, sum(day_demand), k.qty, k.qty - sum(day_demand), k.price_sold*k.profit_margin*(k.qty)
    j += 7

print(dict)

{0: ([21, 23, 21, 22, 22, 23, 22], 154, 100, -54, 2160.0), 1: ([23, 22, 22], 67, 100, 33, -1320.0)}


In [76]:
print(k.demand_distribution()[0])
#print(p.defective_items())

[21, 23, 21, 22, 22, 23, 22, 23, 22, 22]


In [68]:
for i, j in enumerate(p.daily_demand_list[0]):
    print(j)

21
22
22
24
23
23
21
23
21
21


In [52]:
round(100//7)

14